In [1]:
import pandas as pd
import numpy as np
from io import StringIO



In [97]:
def squeeze_nan_left(x):
            original_columns = x.index.tolist()
            squeezed = x.dropna()
            squeezed.index = [original_columns[n] for n in range(squeezed.count())]
            return squeezed.reindex(original_columns, fill_value=np.nan)
    
def filing_table_format(x):
    x = x.dropna(axis = 0, how = 'all').dropna(axis = 1, how = 'all')    
    
    for i in range(0,x.shape[0]):
        for j in range(0,x.shape[1]):
            if type(x.iloc[i,j]) == str:
                if x.iloc[i,j] == "-":
                    x.iloc[i,j] = 0
                elif len(x.iloc[i,j]) < 2:
                    x.iloc[i,j] = np.nan


In [189]:
#Everi - https://www.sec.gov/Archives/edgar/data/1318568/000155837017002779/evri-20170523xdef14a.htm
#MNTA - https://www.sec.gov/Archives/edgar/data/1235010/000123501017000006/mnta2017proxystatement.htm

url = 'https://www.sec.gov/Archives/edgar/data/1235010/000123501017000006/mnta2017proxystatement.htm'

filing = pd.read_html(url)

#168 for Everi
#128 for MNTA

sctind = 128
SCTo = filing[sctind]
SCTo

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Name and Principal Position,Year,Salary(1)($),Stock Awards(2) ($),Option Awards(2) ($),Non‑Equity Incentive Plan Compensation ($),All Other Compensation(3) ($),Total ($),NaN,NaN,NaN,NaN,NaN,NaN
3,Craig A. Wheeler,2016,689179,NaN,2149290,NaN,737818,NaN,554789,NaN,29948.0,NaN,4161024.0,NaN
4,"President, Chief Executive Officer and Director",2015,691484,NaN,3064900,NaN,1046052,NaN,512723,NaN,29503.0,NaN,5344662.0,NaN
5,2014,643356,NaN,1257200,NaN,1558564,NaN,432335,NaN,29610.0,NaN,3921065.0,NaN,NaN
6,Scott M. Storer (4),2016,40385,NaN,684000,NaN,940375,NaN,—,NaN,150.0,NaN,1664910.0,NaN
7,Senior Vice President and Chief Financial Officer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Matthew P. Ottmer (5)(6),2016,450000,NaN,733500,NaN,—,NaN,252281,NaN,13091.0,NaN,1448872.0,NaN


In [202]:
#Table Formatting
#left squeeze needs to apply after for loop

SCT = SCTo.apply(squeeze_nan_left, axis=1).dropna(axis = 0, how = 'all').dropna(axis = 1, how = 'all') 

for i in range(0,SCT.shape[0]):
    for j in range(0,SCT.shape[1]):
        if type(SCT.iloc[i,j]) == str:
            if SCT.iloc[i,j] == "-":
                SCT.iloc[i,j] = 0
            elif '(' in SCT.iloc[i,j]:
                tempstr = SCT.iloc[i,j].split("(")
                tempstr2 = tempstr[0]
                SCT.iloc[i,j] = tempstr2
            elif len(SCT.iloc[i,j]) < 2:
                SCT.iloc[i,j] = np.nan

SCT = SCT.replace('', np.nan)
SCT.iloc[:,0] = SCT.iloc[:,0].fillna(value = '')
SCT = SCT.fillna(value = '')
#SCT = SCT.rename(columns=SCT.iloc[0]).drop(index = 1)                

#SCT.apply(filing_table_format)  

#SCT = SCT.stack()
#type(SCT.iloc[3,4])
SCT

,0,1,2,3,4,5,6,7
2,Name and Principal Position,Year,Salary,Stock Awards,Option Awards,Non‑Equity Incentive Plan Compensation,All Other Compensation,Total
3,Craig A. Wheeler,2016,689179,2149290,737818,554789,29948,4.16102e+06
4,"President, Chief Executive Officer and Director",2015,691484,3064900,1046052,512723,29503,5.34466e+06
5,2014,643356,1257200,1558564,432335,29610,3.92106e+06,
6,Scott M. Storer,2016,40385,684000,940375,,150,1.66491e+06
7,Senior Vice President and Chief Financial Officer,,,,,,,
9,Matthew P. Ottmer,2016,450000,733500,,252281,13091,1.44887e+06
10,Chief Operating Officer,2015,34615,,1507975,,29,1.54262e+06
11,Ganesh V. Kaundinya,2016,433350,791487,221345,201833,13091,1.66111e+06
12,Chief Scientific Officer and Senior Vice Presi...,2015,432708,927139,310065,187924,13014,1.87085e+06


In [196]:
type(SCT.iloc[3,0])

str

In [195]:
SCT.iloc[3,0]

'2014'